In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3


class DeliverySystem(tk.Tk):
    def __init__(self):  # Corrected __init__ method
        super().__init__()
        self.title("Delivery System")
        self.geometry("600x600")
        self.current_user = None

        self.style = ttk.Style(self)
        self.style.theme_use("clam")  # Use a modern theme

        self.setup_database()
        self.setup_login_screen()

    def setup_database(self):
        """Initialize the SQLite database."""
        self.conn = sqlite3.connect("delivery_system.db")
        self.cursor = self.conn.cursor()

        # Create tables if they don't exist
        self.cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE,
                password TEXT,
                role TEXT
            )
            """
        )
        self.cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS items (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT UNIQUE,
                price REAL
            )
            """
        )
        self.cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS orders (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                items TEXT,
                FOREIGN KEY(user_id) REFERENCES users(id)
            )
            """
        )
        # Add initial users
        self.cursor.execute(
            "INSERT OR IGNORE INTO users (username, password, role) VALUES (?, ?, ?)",
            ("admin", "admin123", "admin"),
        )
        self.cursor.execute(
            "INSERT OR IGNORE INTO users (username, password, role) VALUES (?, ?, ?)",
            ("seller", "seller123", "seller"),
        )
        self.cursor.execute(
            "INSERT OR IGNORE INTO users (username, password, role) VALUES (?, ?, ?)",
            ("client1", "client123", "client"),
        )
        self.conn.commit()

    def setup_login_screen(self):
        """Set up the login screen."""
        self.clear_window()

        # Header
        ttk.Label(self, text="Welcome to the Delivery System", font=("Arial", 18)).pack(pady=20)

        # Login Frame
        self.login_frame = ttk.Frame(self)
        self.login_frame.pack(pady=20)

        ttk.Label(self.login_frame, text="Username:").grid(row=0, column=0, padx=10, pady=10)
        self.username_entry = ttk.Entry(self.login_frame)
        self.username_entry.grid(row=0, column=1, padx=10, pady=10)

        ttk.Label(self.login_frame, text="Password:").grid(row=1, column=0, padx=10, pady=10)
        self.password_entry = ttk.Entry(self.login_frame, show="*")
        self.password_entry.grid(row=1, column=1, padx=10, pady=10)

        ttk.Label(self.login_frame, text="Login as:").grid(row=2, column=0, padx=10, pady=10)
        self.role_var = tk.StringVar()
        self.role_menu = ttk.Combobox(self.login_frame, textvariable=self.role_var, state="readonly")
        self.role_menu['values'] = ("admin", "seller", "client")
        self.role_menu.grid(row=2, column=1, padx=10, pady=10)
        self.role_menu.current(0)  # Default to admin

        ttk.Button(self.login_frame, text="Login", command=self.login).grid(row=3, column=0, columnspan=2, pady=20)
        ttk.Button(self.login_frame, text="Register", command=self.register).grid(row=4, column=0, columnspan=2)

    def login(self):
        """Handle login logic."""
        username = self.username_entry.get()
        password = self.password_entry.get()
        selected_role = self.role_var.get()

        self.cursor.execute(
            "SELECT * FROM users WHERE username = ? AND password = ? AND role = ?",
            (username, password, selected_role),
        )
        user = self.cursor.fetchone()

        if user:
            self.current_user = {"id": user[0], "username": user[1], "role": user[3]}
            self.show_dashboard()
        else:
            messagebox.showerror("Login Error", "Invalid username, password, or role.")

    def register(self):
        """Handle user registration."""
        username = self.username_entry.get()
        password = self.password_entry.get()
        selected_role = self.role_var.get()

        try:
            self.cursor.execute(
                "INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                (username, password, selected_role),
            )
            self.conn.commit()
            messagebox.showinfo("Success", f"User registered successfully as {selected_role}.")
        except sqlite3.IntegrityError:
            messagebox.showerror("Registration Error", "User already exists.")

    def show_dashboard(self):
        """Show the appropriate dashboard based on user role."""
        self.clear_window()

        if self.current_user["role"] == "client":
            self.client_dashboard()
        elif self.current_user["role"] == "admin":
            self.admin_dashboard()
        elif self.current_user["role"] == "seller":
            self.seller_dashboard()

    def client_dashboard(self):
        """Set up the client dashboard."""
        ttk.Label(self, text="Available Items", font=("Arial", 18)).pack(pady=10)

        self.items_frame = ttk.Frame(self)
        self.items_frame.pack(pady=10)

        self.item_entries = {}
        self.cursor.execute("SELECT * FROM items")
        all_items = self.cursor.fetchall()

        if not all_items:
            ttk.Label(self.items_frame, text="No items available. Please check back later.").pack()
        else:
            for row, (item_id, item_name, price) in enumerate(all_items, start=1):
                ttk.Label(self.items_frame, text=f"{item_name} - ${price}").grid(row=row, column=0, padx=10, pady=5)
                entry = ttk.Entry(self.items_frame, width=5)
                entry.grid(row=row, column=1, padx=10, pady=5)
                self.item_entries[item_name] = entry

        ttk.Button(self, text="Place Order", command=self.place_order).pack(pady=10)
        ttk.Button(self, text="View My Orders", command=self.view_orders).pack(pady=5)
        ttk.Button(self, text="Logout", command=self.setup_login_screen).pack(pady=10)

    def place_order(self):
        """Place an order for the client."""
        order_items = []
        for item, entry in self.item_entries.items():
            quantity = entry.get()
            if quantity.isdigit() and int(quantity) > 0:
                order_items.append(f"{item} x {quantity}")

        if order_items:
            items_str = ", ".join(order_items)
            self.cursor.execute(
                "INSERT INTO orders (user_id, items) VALUES (?, ?)",
                (self.current_user["id"], items_str),
            )
            self.conn.commit()
            messagebox.showinfo("Success", "Order placed successfully!")
        else:
            messagebox.showwarning("Error", "No valid items selected.")

    def view_orders(self):
        """View and cancel client orders."""
        self.clear_window()

        ttk.Label(self, text="My Orders", font=("Arial", 18)).pack(pady=10)

        self.cursor.execute("SELECT * FROM orders WHERE user_id = ?", (self.current_user["id"],))
        orders = self.cursor.fetchall()

        if not orders:
            ttk.Label(self, text="You have no orders.").pack(pady=20)
        else:
            for order_id, user_id, items in orders:
                ttk.Label(self, text=f"Order #{order_id}: {items}").pack(pady=5)
                ttk.Button(self, text="Cancel", command=lambda oid=order_id: self.cancel_order(oid)).pack()

        ttk.Button(self, text="Back to Dashboard", command=self.client_dashboard).pack(pady=20)

    def cancel_order(self, order_id):
        """Cancel a specific order."""
        self.cursor.execute("DELETE FROM orders WHERE id = ?", (order_id,))
        self.conn.commit()
        messagebox.showinfo("Success", "Order canceled successfully!")
        self.view_orders()

    def seller_dashboard(self):
        """Set up the seller dashboard."""
        ttk.Label(self, text="Seller Dashboard", font=("Arial", 18)).pack(pady=20)

        ttk.Label(self, text="Add New Item").pack(pady=10)

        self.new_item_name = ttk.Entry(self)
        self.new_item_name.pack(pady=5)
        self.new_item_name.insert(0, "Item Name")

        self.new_item_price = ttk.Entry(self)
        self.new_item_price.pack(pady=5)
        self.new_item_price.insert(0, "Item Price")

        ttk.Button(self, text="Add Item", command=self.add_item).pack(pady=10)
        ttk.Button(self, text="View Purchases", command=self.view_purchases).pack(pady=5)
        ttk.Button(self, text="Logout", command=self.setup_login_screen).pack(pady=10)

    def add_item(self):
        """Add a new item to the store."""
        item_name = self.new_item_name.get()
        item_price = self.new_item_price.get()

        if not item_name or not item_price:
            messagebox.showwarning("Error", "Please fill in all fields.")
            return

        try:
            price = float(item_price)
            self.cursor.execute(
                "INSERT INTO items (name, price) VALUES (?, ?)",
                (item_name, price),
            )
            self.conn.commit()
            messagebox.showinfo("Success", f"Item '{item_name}' added successfully.")
        except ValueError:
            messagebox.showerror("Error", "Price must be a valid number.")
        except sqlite3.IntegrityError:
            messagebox.showerror("Error", f"Item '{item_name}' already exists.")

    def view_purchases(self):
        """View purchases made by clients."""
        self.clear_window()

        ttk.Label(self, text="All Purchases", font=("Arial", 18)).pack(pady=10)

        self.cursor.execute("SELECT * FROM orders")
        all_orders = self.cursor.fetchall()

        if not all_orders:
            ttk.Label(self, text="No purchases yet.").pack(pady=20)
        else:
            for order_id, user_id, items in all_orders:
                ttk.Label(self, text=f"Order #{order_id}: {items}").pack(pady=5)

        ttk.Button(self, text="Back to Dashboard", command=self.seller_dashboard).pack(pady=20)

    def admin_dashboard(self):
        """Set up the admin dashboard."""
        ttk.Label(self, text="Admin Dashboard", font=("Arial", 18)).pack(pady=20)

        ttk.Label(self, text="Manage System Users").pack(pady=10)
        ttk.Button(self, text="View Users", command=self.view_users).pack(pady=5)
        ttk.Button(self, text="Logout", command=self.setup_login_screen).pack(pady=10)

    def view_users(self):
        """View and manage system users."""
        self.clear_window()

        ttk.Label(self, text="System Users", font=("Arial", 18)).pack(pady=10)

        self.cursor.execute("SELECT id, username, role FROM users")
        users = self.cursor.fetchall()

        for user_id, username, role in users:
            ttk.Label(self, text=f"User #{user_id}: {username} ({role})").pack(pady=5)

        ttk.Button(self, text="Back to Dashboard", command=self.admin_dashboard).pack(pady=20)

    def clear_window(self):
        """Clear all widgets from the window."""
        for widget in self.winfo_children():
            widget.destroy()

    def on_closing(self):
        """Handle the closing event."""
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            self.conn.close()
            self.destroy()


if __name__ == "__main__":
    app = DeliverySystem()
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()
